## Part II: `Candidate` Extraction

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading the Training `Corpus`

First, we will load the `Corpus` that we preprocessed in Part I:

In [3]:
from snorkel.models import Corpus
from snorkel.utils import get_ORM_instance

corpus = get_ORM_instance(Corpus, session, 'Hardware Training')
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware Training) contains 78 Documents


## Defining a `Candidate` Schema

In [4]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

## Writing a basic `CandidateExtractor`

In [5]:
# from snorkel.matchers import DictionaryMatch

# from hardware_utils import load_extended_parts_dict
# gold_file ='data/hardware/hardware_gold.csv'
# parts_dict = load_extended_parts_dict(gold_file) # NOTE: this include A/B/C/-16/-25/-40 
# print "Loaded %d part numbers." % len(parts_dict)
# parts_matcher = DictionaryMatch(d=parts_dict)

In [6]:
from snorkel.matchers import RegexMatchSpan, Union

# eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,4}[\s]?[A-Z]{0,2}[0-9]?([-][A-Z0-9]{1,3})?)')
# jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]?)')
# jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
# others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,3})?)')
# parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

In [7]:
from snorkel.matchers import RegexMatchSpan

temp_matcher = RegexMatchSpan(rgx=r'-[5-7][05]', longest_match_only=False)

# TEMP

In [8]:
import os
from hardware_utils import get_gold_dict
from collections import defaultdict

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
gold_parts_by_doc = defaultdict(set)
for part in gold_parts:
    gold_parts_by_doc[part[0]].add(part[1])

In [9]:
print len(gold_parts_by_doc)

98


In [10]:
# import codecs
# import csv
# docs = set()

# with codecs.open(gold_file, encoding="utf-8") as csvfile:
#     gold_reader = csv.reader(csvfile)
#     gold_dict = set()
#     i = 0
#     for row in gold_reader:
#         i += 1
#         (doc, part, val, attr) = row
#         docs.add(doc)
# print i
# print len(docs)

In [11]:
# print len(get_gold_dict(gold_file, doc_on=True, part_on=False, val_on=False))

In [12]:
# print len(set([pair[0] for pair in gold_parts]))
# print len(gold_parts_by_doc)
# print gold_parts_by_doc.items()[1]

In [14]:
%time

from snorkel.models import Corpus
from snorkel.utils import get_ORM_instance
from hardware_utils import OmniNgramsPart

corpus = get_ORM_instance(Corpus, session, 'Hardware')
part_ngrams_2 = OmniNgramsPart(n_max=3, split_tokens=None)
parts_by_doc = {}
for doc in corpus.documents:
    for tc in parts_matcher.apply(part_ngrams_2.apply(doc)):
        parts_by_doc[tc.parent.document.name.upper()] = tc.get_span()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


In [15]:
print set(parts_by_doc.keys()).difference(set(gold_parts_by_doc.keys()))
print set(gold_parts_by_doc.keys()).difference(set(parts_by_doc.keys()))

set([])
set(['PJECS00521-1'])


In [16]:
print len(parts_by_doc)

97


In [17]:
print sum(len(x) for x in gold_parts_by_doc.values())

831


In [18]:
print sum(len(x) for x in parts_by_doc.values())

627


In [ ]:
# %time
# fn = []
# for (doc, parts) in gold_parts_by_doc.items():
#     if doc not in parts_by_doc:
#         for part in parts:
#             fn.append((doc, part))
#     else:
#         for part in parts:
#             if part not in parts_by_doc[doc]:
#                 fn.append((doc, part))

In [ ]:
# print len(fn)
# print fn[:10]

# TEMP

In [ ]:
from hardware_utils import OmniNgramsPart, OmniNgramsTemp

part_ngrams = OmniNgramsPart(parts_by_doc=gold_parts_by_doc, n_max=3, split_tokens=None)
temp_ngrams = OmniNgramsTemp(n_max=3, split_tokens=None)

In [ ]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part_Temp, [part_ngrams, temp_ngrams], [parts_matcher, temp_matcher])

## Running the `CandidateExtractor`

In [ ]:
%time train = ce.extract(corpus.documents, 'Hardware Training Candidates', session)
print "%s contains %d Candidates" % (train, len(train))

In [ ]:
for c in train[:3]:
    print c

In [ ]:
# from collections import defaultdict
# from snorkel.utils import ProgressBar

# def get_candidate_id(c):
#     return c.part.get_stable_id() + c.temp.get_stable_id()

# seen = defaultdict(int)
# pb = ProgressBar(len(train))
# for i, c in enumerate(train):
#     pb.bar(i)
#     seen[get_candidate_id(c)] += 1
#     if seen[get_candidate_id(c)] == 2:
#         import pdb; pdb.set_trace()
# pb.close()

In [ ]:
# print type(train[0])

In [ ]:
print len(train) == len(set([c for c in train]))

### Saving the extracted candidates

In [ ]:
session.add(train)
session.commit()

### Reloading the candidates

In [ ]:
from snorkel.models import CandidateSet
from snorkel.utils import get_ORM_instance

train = get_ORM_instance(CandidateSet, session, 'Hardware Training Candidates')
print "%s contains %d Candidates" % (train, len(train))

### Repeating for development and test corpora

In [ ]:
for corpus_name in ['Hardware Development']:
    corpus = get_ORM_instance(Corpus, session, corpus_name)
    print "Extracting Candidates from %s" % corpus
    %time candidates = ce.extract(corpus.documents, corpus_name + ' Candidates', session)
    session.add(candidates)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

In [ ]:
# train = get_ORM_instance(Corpus, session, 'Hardware Training')
# dev = get_ORM_instance(Corpus, session, 'Hardware Development')
# test = get_ORM_instance(Corpus, session, 'Hardware Test')
# trainies = [d.name for d in train.documents]
# len(trainies)
# for d in test.documents:
#     if d.name in trainies:
#         print 'YES!'
# # for d in test.documents[:10]: print d

## TEMPORARY - Assessing Total Recall

In [ ]:
from hardware_utils import entity_level_total_recall
from snorkel.utils import get_ORM_instance

train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()
total_set = set([])
for c in train:
    total_set.add(c)
for c in dev:
    total_set.add(c)

In [ ]:
print list(total_set)[0]

In [ ]:
print len(train)
train_set = set([c for c in train])
print len(train_set)

print len(dev)
dev_set = set([c for c in dev])
print len(dev_set)

print len(total_set)

In [ ]:
# import os
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(total_set, gold_file, relation=True)

In [19]:
from hardware_utils import get_gold_dict
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_attrib = 'stg_temp_min'
gold = gold_dict = get_gold_dict(gold_file, gold_attrib)
print len(gold)

12611


In [22]:
from collections import defaultdict
gold_dict_by_doc = defaultdict(set)
for g in gold_dict:
    gold_dict_by_doc[g[0]].add(g)
print sum([len(gold_dict_by_doc[g]) for g in gold_dict_by_doc])

12611


In [ ]:
# from snorkel.utils import ProgressBar
# target = sorted(list(fn))[-1]
# print target
# print "-------------------------------"
# pb = ProgressBar(len(candidates))
# for i, c in enumerate(list(candidates)[:]):
#     pb.bar(i)
#     if (c.part.parent.document.name.upper() == target[0].upper())
#         and c.part.get_span().upper() == target[1].upper()):
#         print c
# pb.close()
# print len(tp)
# for c in sorted(list(tp))[:5]:
#     print c
# print "-------------------------------"
# print len(fn)
# for c in sorted(list(fn))[:50]:
#     print c

In [ ]:
# from hardware_utils import part_error_analysis

# for c in total_set:
#     if c.part.parent.document.name.upper()=='BC546-D' and c.part.get_span() == 'BC547':
#         part_error_analysis(c)
#         import pdb; pdb.set_trace()

In [ ]:
# corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()

# for doc in corpus.documents:
#     if doc.name == 'PNJIS00254-1':
#         d = doc
#         break
# print d

In [ ]:
# for phrase in d.phrases:
#     if '55' in phrase.words:
#         p = phrase
#         print p.cell
#         import pdb; pdb.set_trace()

In [ ]:
# candies = sorted(candidates, key=lambda x: x[0])

In [ ]:
# from hardware_utils import count_hardware_labels

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# %time count_hardware_labels(candidates, filename, attrib='stg_temp_min', attrib_class='temp')

## TEMPORARY - Return to Normalcy

In [ ]:
# If necessary
# import os
# os.system('cp snorkel.db snorkel.db\ candidates');

Next, in Part 3, we will load `Labels` for each of our `Candidates` so that we can evaluate performance.